In [ ]:
import torch
import pickle

In [ ]:
with open("../feed_dict.json","rb") as file:
    feed_dict = pickle.load(file)

In [ ]:
feed_dict_q = feed_dict
feed_dict_k = feed_dict

feed_dict_q["view"] = torch.rand((feed_dict["image"].size(0),1,7))
feed_dict_k["view"] = torch.rand((feed_dict["image"].size(0),1,7))

In [ ]:
import sys
sys.path.append("../")

In [ ]:
# default_exp model.main

In [ ]:
#export

from torchvision.models import resnet34
import torch.nn as nn
import torch

from core.scene_graph.scene_graph import SceneGraph

In [ ]:
#export

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
    
        self.resnet = resnet34(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-3])
        self.scene_graph = SceneGraph(feature_dim=256, 
                                 output_dims=[256,256],
                                 downsample_rate=16)
        
        self.visual_view_point_transformation_layer = nn.Sequential(nn.Linear(263,512),
                                                nn.ReLU(),
                                                nn.Linear(512,263))

        self.spatial_view_point_transformation_layer = nn.Sequential(nn.Linear(263,512),
                                                        nn.ReLU(),
                                                        nn.Linear(512,263))

        ##### To DO : Reset Parameters of "view_point_transformation_layer"
        self.reset_parameters()
        
    def forward(self,feed_dict_k, feed_dict_q):
    
        scene_embeddings_q = self.get_scene_graph(feed_dict_q)
        scene_embeddings_k = self.get_scene_graph(feed_dict_k)
        
        scene_embeddings_q = merge_pose_with_scene_embeddings(scene_embeddings_q, feed_dict_q["view"])
        scene_embeddings_k = merge_pose_with_scene_embeddings(scene_embeddings_k, feed_dict_k["view"])
        
        scene_embeddings_k_transformed = do_viewpoint_transformation(scene_embeddings_k)
        
        return scene_embeddings_q, scene_embeddings_q, scene_embeddings_k_transformed
    

    def get_scene_graph(self,feed_dict):
        image_features = self.feature_extractor(feed_dict["image"])
        outputs = self.scene_graph(image_features, feed_dict["objects_boxes"], feed_dict["objects"])

        return outputs
    
    @staticmethod
    def merge_pose_with_scene_embeddings(scene_embeddings, view):

        for ind,(visual_embeddings, spatial_embeddings) in enumerate(scene_embeddings):

            # Broadcast view to visual embedding dimension
            view_visual = view[ind].repeat(visual_embeddings.shape[0],1)
            # Concatenate with visual embeddings
            pose_with_features = torch.cat((view_visual,visual_embeddings), dim=1)
            # Reassign the scene embeddings
            scene_embeddings[ind][0] = pose_with_features

            # Broadcast view to spatial embedding dimension
            view_spatial = view[ind].unsqueeze(0).repeat(spatial_embeddings.shape[0],spatial_embeddings.shape[1],1)
            # Concatenate with visual embeddings
            pose_with_features = torch.cat((view_spatial,spatial_embeddings), dim=2)
            # Reassign the scene embeddings
            scene_embeddings[ind][1] = pose_with_features

            ### To Do : Write some assertion test : (Saksham)

        return scene_embeddings
    
    @staticmethod  
    def do_viewpoint_transformation(scene_embeddings):
    
        for ind,(visual_embeddings, spatial_embeddings) in enumerate(scene_embeddings):

            # Do viewpoint transformation on visual embeddings
            scene_embeddings[ind][0] = visual_view_point_transformation_layer(scene_embeddings_k[ind][0])

            # Do viewpoint transformation on spatial embeddings
            scene_embeddings[ind][1] = spatial_view_point_transformation_layer(scene_embeddings_k[ind][1])

        return scene_embeddings
    
    def reset_parameters(self):
        pass

In [ ]:
m = Model()

In [ ]:
outputs = m(feed_dict_k, feed_dict_q)